可转债上市提醒 (可运行)

http://data.eastmoney.com/xg/kzz/default.html

In [37]:
from selenium import webdriver
from pyquery import PyQuery as pq
import easygui as eg
from threading import Timer
import datetime

def bond_check(driver,url,bond_no):
    driver.get(url) 
    html = driver.page_source
    doc = pq(html,parser="html")
    content = doc('.maincont tbody tr')
    for item in content.items():
        bond_code = int(item.find("td:first-child").text())
        if bond_no == bond_code:
            bond_title = item.find("td:nth-child(2)").text()
            bond_rate = item.find("td:nth-child(13)").text()
            list_status = item.find("td:last-child").text()
            message = '债券代码:{},债券简称:{},溢价率:{},上市时间:{}'.format(bond_code,bond_title,bond_rate,list_status)  
            if list_status != '-':
                print('{}上市提醒'.format(bond_title))
                reminder(message)
                bond_list.remove(bond_no)
            else:
                print('{}未上市'.format(bond_title))
            print(message)
            
def reminder(message):
    eg.msgbox(message,'上市提醒',ok_button='了解')
    
def main(url,bond_list):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    print('{} 债券上市状态查询开始'.format(datetime.datetime.now()))
    for bond_no in bond_list:
        bond_check(driver,url,bond_no)
    print('本轮债券上市状态查询结束')

if __name__ == '__main__': 
    url = 'http://data.eastmoney.com/xg/kzz/default.html'
    bond_list = [127012, 123025, 128062, 128064, 113025]
    main(url,bond_list)
    print('24小时候继续查询......')
    timer = Timer(86400,main(url,bond_list))
    timer.start()

2019-04-02 12:04:03.869753 债券上市状态查询开始
招路转债未上市
债券代码:127012,债券简称:招路转债,溢价率:2.98%,上市时间:-
精测转债未上市
债券代码:123025,债券简称:精测转债,溢价率:-13.66%,上市时间:-
本轮债券上市状态查询结束
24小时候继续查询......
2019-04-02 12:04:17.365750 债券上市状态查询开始
招路转债未上市
债券代码:127012,债券简称:招路转债,溢价率:2.98%,上市时间:-
精测转债未上市
债券代码:123025,债券简称:精测转债,溢价率:-13.66%,上市时间:-
本轮债券上市状态查询结束


## 多线程

In [6]:
from selenium import webdriver
from pyquery import PyQuery as pq
import easygui as eg
from threading import Timer
import datetime
from threading import Thread

class Bond_check(Thread):
    
    def __init__(self,driver,url,bond_no):
        super().__init__()
        self.driver = driver
        self.url = url
        self.bond_no = bond_no
        
    def run(self):
        self.driver.get(self.url) 
        html = self.driver.page_source
        doc = pq(html,parser="html")
        content = doc('.maincont tbody tr')
        for item in content.items():
            bond_code = int(item.find("td:first-child").text())
            if self.bond_no == bond_code:
                bond_title = item.find("td:nth-child(2)").text()
                bond_rate = item.find("td:nth-child(13)").text()
                list_status = item.find("td:last-child").text()
                message = '债券代码:{},债券简称:{},溢价率:{},上市时间:{}'.format(bond_code,bond_title,bond_rate,list_status)  
                if list_status != '-':
                    bond_list.remove(self.bond_no)
                    print('{}上市提醒'.format(bond_title))
                    reminder(message)
                    print(message)
                    
                else:
                    print('{}未上市'.format(bond_title))
                print(message)
            
def reminder(message):
    eg.msgbox(message,'上市提醒',ok_button='了解')
    
def main(url,bond_list):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    print('{} 债券上市状态查询开始'.format(datetime.datetime.now()))
    threads = [Bond_check(driver,url,bond_no) for bond_no in bond_list]
    for t in threads:
        t.start()
    for t in threads:    
        t.join()
    print('本轮债券上市状态查询结束')

if __name__ == '__main__': 
    url = 'http://data.eastmoney.com/xg/kzz/default.html'
    bond_list = [127012, 123025, 128062, 128064, 113025]
    main(url,bond_list)
    print('24小时候继续查询......')
    timer = Timer(86400,main(url,bond_list))
    timer.start()

2019-04-19 10:51:23.717067 债券上市状态查询开始
招路转债未上市
债券代码:127012,债券简称:招路转债,溢价率:5.42%,上市时间:-
精测转债上市提醒
明泰转债未上市
债券代码:113025,债券简称:明泰转债,溢价率:3.51%,上市时间:-
亚药转债未上市
债券代码:128062,债券简称:亚药转债,溢价率:-11.51%,上市时间:-
司尔转债未上市
债券代码:128064,债券简称:司尔转债,溢价率:-8.36%,上市时间:-
本轮债券上市状态查询结束


Exception in thread Thread-12:
Traceback (most recent call last):
  File "c:\users\win7\appdata\local\programs\python\python36-32\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-6-0f753b465e17>", line 31, in run
    bond_list.remove(bond_no)
NameError: name 'bond_no' is not defined



##  获取新债上市首日表现 
### 数据来源：东方财富移动端

In [33]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time, random, re
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import pymongo



# Appium 基本参数
PLATFORM = 'Android'
DEVICE_NAME = 'MI_PAD_4'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 800
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700

class Automation():
    # 初始化 Appium 基本参数
    def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.desired_caps = {
            'platformName': PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER, self.desired_caps)
        self.wait = WebDriverWait(self.driver, 10)
        self.size = self.driver.get_window_size()


# 屏幕方法
    def scroll_bottom(self):
        # 滑动到底部
        print('滑动到底部')
        Swiped = False
        # 获取滑动前页面元素
        while not Swiped:
            before_swipe = self.driver.page_source
            # 滑动
            self.driver.swipe(600, 1600, 630, 100) 
            # 获取滑动后页面元素
            after_swipe = self.driver.page_source
            # 对比滑动前后的页面元素
            # 若滑动前后页面元素完全相同，则滑动失败，已达页尾
            if before_swipe == after_swipe:
                Swiped = False
            # 若滑动前后页面元素不同，则滑动成功，继续滑动
            else:
                Swiped = True


    def swipeUp(self):
        # 向上滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.75, 0.85),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.15, 0.35), random.uniform(800, 1200))
        print('向上滑动屏幕')

    def swipeDown(self):
        # 向下滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.25, 0.35),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.65, 0.75), random.uniform(800, 1200))
        print('向下滑动屏幕')

    def swipeRight(self):
        # 向右滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.01, 0.11),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.89, 0.98),
                          self.size['height'] * random.uniform(0.75, 0.89), random.uniform(800, 1200))
        print('向右滑动屏幕')

class Eastmoney(Automation):
    
    def __init__(self,APP_PACKAGE, APP_ACTIVITY):
        # 东方财富操作元素
        self.transaction_ele = 'com.eastmoney.android.berlin:id/btn5'
        self.login_ele = '//*[@text="登录"]'
        self.psw_ele = 'com.eastmoney.android.berlin:id/password_tv'
        self.enter_ele = 'com.eastmoney.android.berlin:id/trade_login_btn'
        self.applybond_ele = '//*[@text="可转债申购"]'
        self.performance_ele = 'subNav3'
        self.bonditem_ele = 'mpi'
        self.load_ele = 'loadMarketPerform'
        super().__init__(APP_PACKAGE, APP_ACTIVITY)
        
    def login(self): 
        print('打开 {} App ...'.format('东方财富'))
        print('等待6秒钟...')
        time.sleep(6)
        print('点击底部交易按钮...')
        trans_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.transaction_ele)))
        trans_btn.click()
        print('点击登录按钮...')     
        login_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.login_ele)))
        login_btn.click()
        print('输入密码...')
        psw_input = self.wait.until(EC.presence_of_element_located((By.ID,self.psw_ele)))
        psw_input.send_keys("539020")
        print('登录中...')
        enter_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.enter_ele)))
        enter_btn.click()    
        
    def bond(self):
        print('点击可转债申购...')
        applybond_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.applybond_ele)))
        applybond_btn.click()        
        print('点击上市表现...')
        performance_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.performance_ele )))
        performance_btn.click() 

        
    def crawl(self):
        print('获取可转债数据')
        client = pymongo.MongoClient(host='localhost', port=27017)
        db = client.eastmoney
        collection = db.bonds
        page = 0
        while True:
            print('第{}页'.format(page+1))
            items = self.driver.find_elements_by_id(self.bonditem_ele)
            bond_list = []
            print('可转债个数:',len(items))
            for item in items[page*20:]:
                eles = item.find_elements_by_xpath('//android.view.View')
                bond_dict = dict()
                bond_dict['bond_name'] = eles[1].get_attribute('text')
                bond_dict['bond_code'] = eles[2].get_attribute('text')
                bond_dict['list_date'] = eles[3].get_attribute('text')
                bond_dict['current_price'] = eles[4].get_attribute('text')
                bond_dict['percent'] = eles[5].get_attribute('text')
                bond_dict['first_day'] = eles[6].get_attribute('text')
                bond_list.append(bond_dict)
            print(bond_list)
            if page>2 :
                print('存入数据库中...')
                collection.insert_many(bond_list) 
            self.scroll_bottom()
            while bond_list = 0:
                self.scroll_bottom()
            load_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.load_ele)))
            print('点击加载更多')
            load_btn.click() 
            time.sleep(3)
            page += 1    
        
    def main(self):
        self.login()
        self.bond()
        self.crawl()

eastmoney_appPackage = 'com.eastmoney.android.berlin'
eastmoney_appActivity = 'com.eastmoney.android.berlin.activity.MainActivity'


if __name__ == '__main__':
    eastmoney = Eastmoney(eastmoney_appPackage,eastmoney_appActivity)
    eastmoney.main()

打开 appium 服务器...
配置 appium ...
打开 东方财富 App ...
等待6秒钟...
点击底部交易按钮...
点击登录按钮...
输入密码...
登录中...
点击可转债申购...
点击上市表现...
获取可转债数据
第1页
可转债个数: 20
[{'bond_name': '恩捷转债', 'bond_code': '128095', 'list_date': '02-28', 'current_price': '127.22', 'percent': '27.22%', 'first_day': '292.23元'}, {'bond_name': '柳药转债', 'bond_code': '113563', 'list_date': '02-24', 'current_price': '125.63', 'percent': '25.63%', 'first_day': '247.55元'}, {'bond_name': '星帅转债', 'bond_code': '128094', 'list_date': '02-19', 'current_price': '116.58', 'percent': '16.58%', 'first_day': '184.25元'}, {'bond_name': '博威转债', 'bond_code': '113031', 'list_date': '02-17', 'current_price': '134.77', 'percent': '34.77%', 'first_day': '281.07元'}, {'bond_name': '银河转债', 'bond_code': '123042', 'list_date': '02-14', 'current_price': '115.18', 'percent': '15.18%', 'first_day': '126.44元'}, {'bond_name': '东财转2', 'bond_code': '123041', 'list_date': '02-14', 'current_price': '153.80', 'percent': '53.80%', 'first_day': '297.11元'}, {'bond_name': '正裕转债', '

KeyboardInterrupt: 

In [ ]:
import pymongo

client = pymongo.MongoClient(host='localhost', port=27017)
db = client.eastmoney
collection = db.bonds
collection.insert_many([student1, student2])

In [40]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time, random, re
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import pymongo



# Appium 基本参数
PLATFORM = 'Android'
DEVICE_NAME = 'MI_PAD_4'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 800
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700

class Automation():
    # 初始化 Appium 基本参数
    def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.desired_caps = {
            'platformName': PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER, self.desired_caps)
        self.wait = WebDriverWait(self.driver, 10)
        self.size = self.driver.get_window_size()


# 屏幕方法
    def swipeUp(self):
        # 向上滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.90, 1),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.1, 0.25))
        print('向上滑动屏幕')

    def swipeDown(self):
        # 向下滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.25, 0.35),
                          self.size['width'] * random.uniform(0.55, 0.65),
                          self.size['height'] * random.uniform(0.65, 0.75))
        print('向下滑动屏幕')

    def swipeRight(self):
        # 向右滑动屏幕
        self.driver.swipe(self.size['width'] * random.uniform(0.01, 0.11),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.89, 0.98),
                          self.size['height'] * random.uniform(0.75, 0.89))
        print('向右滑动屏幕')

class Eastmoney(Automation):
    
    def __init__(self,APP_PACKAGE, APP_ACTIVITY):
        # 东方财富操作元素
        self.transaction_ele = 'com.eastmoney.android.berlin:id/btn5'
        self.login_ele = '//*[@text="登录"]'
        self.psw_ele = 'com.eastmoney.android.berlin:id/password_tv'
        self.enter_ele = 'com.eastmoney.android.berlin:id/trade_login_btn'
        self.applybond_ele = '//*[@text="可转债申购"]'
        self.performance_ele = 'subNav3'
        self.bonditem_ele = 'mpi'
        self.load_ele = 'loadMarketPerform'
        super().__init__(APP_PACKAGE, APP_ACTIVITY)
        
    def login(self): 
        print('打开 {} App ...'.format('东方财富'))
        print('等待6秒钟...')
        time.sleep(6)
        print('点击底部交易按钮...')
        trans_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.transaction_ele)))
        trans_btn.click()
        print('点击登录按钮...')     
        login_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.login_ele)))
        login_btn.click()
        print('输入密码...')
        psw_input = self.wait.until(EC.presence_of_element_located((By.ID,self.psw_ele)))
        psw_input.send_keys(password)
        print('登录中...')
        enter_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.enter_ele)))
        enter_btn.click()    
        
    def bond(self):
        print('点击可转债申购...')
        applybond_btn = self.wait.until(EC.presence_of_element_located((By.XPATH,self.applybond_ele)))
        applybond_btn.click()        
        print('点击上市表现...')
        performance_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.performance_ele )))
        performance_btn.click() 

        
    def crawl(self):
        print('获取可转债数据')
        client = pymongo.MongoClient(host='localhost', port=27017)
        db = client.eastmoney
        collection = db.bonds
        page = 0
        while True:
            self.swipeUp()
            print('第{}页'.format(page+1))
            items = self.driver.find_elements_by_id(self.bonditem_ele)
            bond_list = []
            print('可转债个数:',len(items))
#             while len(items) == 0:
#                 self.swipeUp()
#                 load_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.load_ele)))
#                 print('点击加载更多')
#                 load_btn.click() 
#                 items = self.driver.find_elements_by_id(self.bonditem_ele)
            if page>8 :    
                for item in items[page*20:]:
                    eles = item.find_elements_by_xpath('//android.view.View')
                    bond_dict = dict()
                    bond_dict['bond_name'] = eles[1].get_attribute('text')
                    bond_dict['bond_code'] = eles[2].get_attribute('text')
                    bond_dict['list_date'] = eles[3].get_attribute('text')
                    bond_dict['current_price'] = eles[4].get_attribute('text')
                    bond_dict['percent'] = eles[5].get_attribute('text')
                    bond_dict['first_day'] = eles[6].get_attribute('text')
                    bond_list.append(bond_dict)
                print(bond_list)
                print('存入数据库中...')
                collection.insert_many(bond_list) 
            self.swipeUp()
            load_btn = self.wait.until(EC.presence_of_element_located((By.ID,self.load_ele)))
            print('点击加载更多')
            load_btn.click() 
            time.sleep(3)
            page += 1    
        
    def main(self):
        self.login()
        self.bond()
        self.crawl()

eastmoney_appPackage = 'com.eastmoney.android.berlin'
eastmoney_appActivity = 'com.eastmoney.android.berlin.activity.MainActivity'


if __name__ == '__main__':
    eastmoney = Eastmoney(eastmoney_appPackage,eastmoney_appActivity)
    eastmoney.main()

打开 appium 服务器...
配置 appium ...
打开 东方财富 App ...
等待6秒钟...
点击底部交易按钮...
点击登录按钮...
输入密码...
登录中...
点击可转债申购...
点击上市表现...
获取可转债数据
向上滑动屏幕
第1页
可转债个数: 20
向上滑动屏幕
点击加载更多
向上滑动屏幕
第2页
可转债个数: 40
向上滑动屏幕
点击加载更多
向上滑动屏幕
第3页
可转债个数: 60
向上滑动屏幕
点击加载更多
向上滑动屏幕
第4页
可转债个数: 80
向上滑动屏幕
点击加载更多
向上滑动屏幕
第5页
可转债个数: 100
向上滑动屏幕
点击加载更多
向上滑动屏幕
第6页
可转债个数: 120
向上滑动屏幕
点击加载更多
向上滑动屏幕
第7页
可转债个数: 140
向上滑动屏幕
点击加载更多
向上滑动屏幕
第8页
可转债个数: 160
向上滑动屏幕
点击加载更多
向上滑动屏幕
第9页
可转债个数: 180
向上滑动屏幕
点击加载更多
向上滑动屏幕
第10页
可转债个数: 200
[{'bond_name': '星源转债', 'bond_code': '123009', 'list_date': '04-10', 'current_price': '126.49', 'percent': '26.49%', 'first_day': '203.13元'}, {'bond_name': '金农转债', 'bond_code': '128036', 'list_date': '04-02', 'current_price': '124.50', 'percent': '24.50%', 'first_day': '2.92元'}, {'bond_name': '杭电转债', 'bond_code': '113505', 'list_date': '03-27', 'current_price': '109.33', 'percent': '9.33%', 'first_day': '9.21元'}, {'bond_name': '艾华转债', 'bond_code': '113504', 'list_date': '03-23', 'current_price': '131.26', 'percent': '31.26%', 'fir

TypeError: documents must be a non-empty list

# 债券-沪深可转债
 "bond_zh_hs_cov_daily"  # 债券-沪深可转债-历史行情数据
 "bond_zh_hs_cov_spot"  # 债券-沪深可转债-实时行情数据

In [5]:
import akshare as ak
bond_zh_hs_cov_spot_df = ak.bond_zh_hs_cov_spot()
print(type(bond_zh_hs_cov_spot_df))


1
2
3
<class 'pandas.core.frame.DataFrame'>
